In [24]:
#!pip install torch accelerate torchaudio datasets
#!pip install --upgrade transformers
#!pip install git+https://github.com/huggingface/transformers.git

In [4]:
from transformers import Wav2Vec2ForCTC, AutoProcessor
import torch
import librosa
import numpy as np

model_id = "facebook/mms-1b-all"

processor = AutoProcessor.from_pretrained(model_id)
model = Wav2Vec2ForCTC.from_pretrained(model_id)

In [25]:
df_utterances = pd.read_csv('/content/gautam-lang-final.csv')
df_utterances.head()

,Unnamed: 0,hi,tr,pt,hu,id,en,es
0,0,"इस वर ् ष के अंत तक , इस ग ् रह पर लगभग एक अरब...","Bu senenin sonunda , yaklaşık bir milyar insan...","Até ao final deste ano , existirão quase um mi...",Az év végéig majdnem egymilliárd ember lesz a ...,"Pada akhir tahun ini , akan ada hampir satu mi...","By the end of this year , there &apos;ll be ne...",A finales de este año habrá alrededor de mil m...
1,1,उन सब में जो एक बात आम है वह है कि वे मरने वाल...,"Hepsinin tek ortak noktası da , ölecek olmaları .",A única coisa que todas elas têm em comum é qu...,"És ami mindannyiukban közös , hogy mind meg fo...",Satu kesamaan dari mereka semua adalah mereka ...,The one thing that all of them have in common ...,Todos tienen en común que van a morir .
2,2,"हालांकि यह कुछ हद तक एक रुग ् ण सोच है , मुझे ...","Bu biraz iğrenç bir düşünce olsa da , içinde k...",Apesar de isso poder ser um pensamento um tant...,"Lehet , hogy ez egy morbid gondolat , de azt h...","Walau itu pemikiran yang agak menakutkan , say...",While that might be a somewhat morbid thought ...,"Aunque esto pueda ser un poco morboso , creo q..."
3,3,मुझे पहली बार इसका ख ् याल आया डेरेक के.एच. मि...,"Beni bunu düşünmeye iten şey , Derek K. Miller...","Portanto , o que me fez começar a pensar acerc...",Egy blog bejegyzés miatt kezdtem el ezen gondo...,Yang membuat saya mulai berpikir seperti ini a...,What first got me thinking about this was a bl...,Lo primero que me llevó a pensar en esto fue u...
4,4,और मिलर ने अपने परिवार और दोस ् तों से एक पोस ...,Miller ölümünden kısa bir süre sonra paylaşılm...,"E o que Miller fez , foi fazer com que a seu f...","Miller megkérte a családját és barátait , hogy...",Miller meminta keluarga dan temannya untuk men...,And what Miller did was have his family and fr...,Miller le pidió a sus familiares y amigos que ...


In [35]:
pd_transcriptions = {
    'hindi_orig':[],
    'hindi_trans':[],
    'english_orig':[],
    'english_trans':[],
    'hungarian_orig':[],
    'hungarian_trans':[],
    'indonesian_orig':[],
    'indonesian_trans':[],
    'portugese_orig':[],
    'portugese_trans':[],
    'spanish_orig':[],
    'spanish_trans':[],
    'turkish_orig':[],
    'turkish_trans':[],
}

base_path = '/content/utterance'

for index, row in df_utterances.iterrows():

  hi = row['hi']
  tr = row['tr']
  pt = row['pt']
  hu = row['hu']
  id = row['id']
  en = row['en']
  es = row['es']

  folder_path = base_path + str(index+1) + '/'
  audio_file_english = folder_path + 'eng.wav'
  audio_file_hindi = folder_path + 'hin.wav'
  audio_file_turkish = folder_path + 'tur.wav'
  audio_file_portugese = folder_path + 'por.wav'
  audio_file_hungarian = folder_path + 'hun.wav'
  audio_file_indonesian = folder_path + 'ind.wav'
  audio_file_spanish = folder_path + 'spa.wav'

  #English

  audio_english, sr = librosa.load(audio_file_english, sr=None)

  inputs = processor(audio_english, sampling_rate=16_000, return_tensors="pt")

  with torch.no_grad():
      outputs = model(**inputs).logits

  ids = torch.argmax(outputs, dim=-1)[0]
  transcription_english = processor.decode(ids)

  #Hindi

  audio_hindi, sr = librosa.load(audio_file_hindi, sr=None)

  processor.tokenizer.set_target_lang("hin")
  model.load_adapter("hin")

  inputs = processor(audio_hindi, sampling_rate=16_000, return_tensors="pt")

  with torch.no_grad():
      outputs = model(**inputs).logits

  ids = torch.argmax(outputs, dim=-1)[0]
  transcription_hindi = processor.decode(ids)

  print(transcription_english,en)
  print(transcription_hindi,hi)

  #Turkish

  audio_turkish, sr = librosa.load(audio_file_turkish, sr=None)

  processor.tokenizer.set_target_lang("tur")
  model.load_adapter("tur")

  inputs = processor(audio_turkish, sampling_rate=16_000, return_tensors="pt")

  with torch.no_grad():
      outputs = model(**inputs).logits

  ids = torch.argmax(outputs, dim=-1)[0]
  transcription_turkish = processor.decode(ids)

  #Hungarian

  audio_hungarian, sr = librosa.load(audio_file_hungarian, sr=None)

  processor.tokenizer.set_target_lang("hun")
  model.load_adapter("hun")

  inputs = processor(audio_hungarian, sampling_rate=16_000, return_tensors="pt")

  with torch.no_grad():
      outputs = model(**inputs).logits

  ids = torch.argmax(outputs, dim=-1)[0]
  transcription_hungarian = processor.decode(ids)

  #Portugese

  audio_portugese, sr = librosa.load(audio_file_portugese, sr=None)

  processor.tokenizer.set_target_lang("por")
  model.load_adapter("por")

  inputs = processor(audio_portugese, sampling_rate=16_000, return_tensors="pt")

  with torch.no_grad():
      outputs = model(**inputs).logits

  ids = torch.argmax(outputs, dim=-1)[0]
  transcription_portugese = processor.decode(ids)

  #Spanish

  audio_spanish, sr = librosa.load(audio_file_spanish, sr=None)

  processor.tokenizer.set_target_lang("spa")
  model.load_adapter("spa")

  inputs = processor(audio_spanish, sampling_rate=16_000, return_tensors="pt")

  with torch.no_grad():
      outputs = model(**inputs).logits

  ids = torch.argmax(outputs, dim=-1)[0]
  transcription_spanish = processor.decode(ids)

  #Indonesian

  audio_indonesian, sr = librosa.load(audio_file_indonesian, sr=None)

  processor.tokenizer.set_target_lang("ind")
  model.load_adapter("ind")

  inputs = processor(audio_indonesian, sampling_rate=16_000, return_tensors="pt")

  with torch.no_grad():
      outputs = model(**inputs).logits

  ids = torch.argmax(outputs, dim=-1)[0]
  transcription_indonesian = processor.decode(ids)


  print(transcription_english,en)
  print(transcription_hindi,hi)
  print(transcription_turkish,tr)
  print(transcription_hungarian,hu)
  print(transcription_portugese,pt)
  print(transcription_spanish,es)
  print(transcription_indonesian,id)

  pd_transcriptions['hindi_orig'].append(hi)
  pd_transcriptions['hindi_trans'].append(transcription_hindi)

  pd_transcriptions['english_orig'].append(en)
  pd_transcriptions['english_trans'].append(transcription_english)

  pd_transcriptions['hungarian_orig'].append(hu)
  pd_transcriptions['hungarian_trans'].append(transcription_hungarian)
  
  pd_transcriptions['indonesian_orig'].append(id)
  pd_transcriptions['indonesian_trans'].append(transcription_indonesian)

  pd_transcriptions['portugese_orig'].append(pt)
  pd_transcriptions['portugese_trans'].append(transcription_portugese)
  
  pd_transcriptions['turkish_orig'].append(tr)
  pd_transcriptions['turkish_trans'].append(transcription_turkish)

  pd_transcriptions['spanish_orig'].append(es)
  pd_transcriptions['spanish_trans'].append(transcription_spanish)

by the end of this year their apostle be nearly a billion people on this planet that actively use social networking sites By the end of this year , there &apos;ll be nearly a billion people on this planet that actively use social networking sites .
इस वर्क्ष के अम तक इसग कह पर लगभग एक अरब लोग ऐसे होंगे जो सक क्रिय रूप से सामाजिक नेटव और किमम साइटों का उपयोग करेंगे इस वर ् ष के अंत तक , इस ग ् रह पर लगभग एक अरब लोग ऐसे होंगे जो सक ् रिय रूप से सामाजिक नेटवर ् किंग साइटों का उपयोग करेंगे .


by the end of this year their apostle be nearly a billion people on this planet that actively use social networking sites By the end of this year , there &apos;ll be nearly a billion people on this planet that actively use social networking sites .
इस वर्क्ष के अम तक इसग कह पर लगभग एक अरब लोग ऐसे होंगे जो सक क्रिय रूप से सामाजिक नेटव और किमम साइटों का उपयोग करेंगे इस वर ् ष के अंत तक , इस ग ् रह पर लगभग एक अरब लोग ऐसे होंगे जो सक ् रिय रूप से सामाजिक नेटवर ् किंग साइटों का उपयोग करेंगे .
bu senenin sonunda yaklaşık bir milyar insan sosyal asitererini aktif olarak kullanıyor olacak Bu senenin sonunda , yaklaşık bir milyar insan sosyal ağ sitelerini aktif olarak kullanıyor olacak .
az év végég majdnem egy milliárd dembe lesz a bolygóm aki aktívan használ közösségi oldalakat Az év végéig majdnem egymilliárd ember lesz a bolygón , aki aktívan használ közösségi oldalakat .
até ao final deste ano existirão quase um milhão de pessoas neste planeta que utilizam ativamente páginas de redes soci

In [36]:
pd_transcriptions = pd.DataFrame(pd_transcriptions)

In [37]:
pd_transcriptions.to_csv('utterances_to_transcriptions.csv')

In [38]:
from google.colab import files

files.download('utterances_to_transcriptions.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [14]:
inputs = processor(audio, sampling_rate=16_000, return_tensors="pt")

with torch.no_grad():
    outputs = model(**inputs).logits

ids = torch.argmax(outputs, dim=-1)[0]
transcription = processor.decode(ids)
# 'joe keton disapproved of films and buster also had reservations about the media'

In [16]:
import pandas as pd
df_lang = pd.read_csv('/content/gautam-lang-final.csv')

In [18]:
first_row = df_lang.iloc[0]
#indexed_row = first_row['Name':'City']
first_row['en']

'By the end of this year , there &apos;ll be nearly a billion people on this planet that actively use social networking sites .'

In [15]:
transcription

'by the end of this year their apostle be nearly a billion people on this planet that actively use social metworking sites'

In [20]:
!pip install jiwer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 24.2 MB/s eta 0:00:00


In [21]:
from jiwer import wer

reference = first_row['en']
hypothesis = transcription

error = wer(reference, hypothesis)

In [22]:
error

0.25

In [ ]:
processor.tokenizer.set_target_lang("fra")
model.load_adapter("fra")

inputs = processor(fr_sample, sampling_rate=16_000, return_tensors="pt")

with torch.no_grad():
    outputs = model(**inputs).logits

ids = torch.argmax(outputs, dim=-1)[0]
transcription = processor.decode(ids)
# "ce dernier est volé tout au long de l'histoire romaine"

In [ ]:
processor.tokenizer.vocab.keys()